# Time Series Anomaly Detection

In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
import seaborn as sns
import env


The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.
Hint: You will need to explore the database (yay SQL!)

1. Label students by the program they are in.

2. Is it possible to identify the user_id of a staff member?

3. Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

4. Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2.

5. Plot the access activity of these students.

BONUS: Can you identify users who are viewing both the web dev and data science curriculum?

### Acquire

In [2]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = '''
SELECT *
FROM logs
LEFT JOIN cohorts ON cohorts.id= logs.cohort_id
'''
df = pd.read_sql(query, url)
df.head(20)

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0
5,2018-01-26,09:56:41,javascript-i/loops,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0
6,2018-01-26,09:56:46,javascript-i/conditionals,3,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0
7,2018-01-26,09:56:48,javascript-i/functions,3,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0
8,2018-01-26,09:56:59,javascript-i/loops,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0
9,2018-01-26,09:58:26,javascript-i/functions,4,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0


In [3]:
df.start_date = pd.to_datetime(df.start_date)
df.end_date = pd.to_datetime(df.end_date)
df.created_at = pd.to_datetime(df.created_at)
df.updated_at = pd.to_datetime(df.updated_at)
df.date = pd.to_datetime(df.date)
    # sets index as dt object
df = df.set_index(df.date)
    # creates column caled pages that is the number of paths per day
pages = df['path'].resample('d').count()
    # name changes
df['program_name']= df.program_id
df.rename(columns={'path':'endpoint'}, inplace = True)
    # making programs human-readable
df.program_name = df.program_name.replace({1.0:'full_stack_php',2.0:'full_stack_java',3.0:'data_science',4.0:'front_end'})
df['program_type'] = df.program_name.rename({'full_stack_php':'wd', 'full_stack_java':'wd','front_end':'wd','data_science':'ds'})
    # elimating staff from df
df = df[df.name != 'Staff']
    
df.head()




TypeError: rename() got an unexpected keyword argument 'columns'

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.head(20)

In [ ]:
df['program_type'] = df.program_name.rename({'full_stack_php':'wd', 'full_stack_java':'wd','front_end':'wd','data_science':'ds'})

In [ ]:
df.head()

In [ ]:
df[df.cohort_id.isnull()]

### Prepare

In [ ]:
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)
pages = df['path'].resample('d').count()
pages.head()

### Label Program Name

In [ ]:
df.program_id.value_counts()

In [ ]:
df['program_name']= df.program_id

In [ ]:
df.program_name = df.program_name.replace({1.0:'full_stack_php',2.0:'full_stack_java',3.0:'data_science',4.0:'front_end'})

In [ ]:
df

In [ ]:
df.to_csv('curriculum_logs_edited', index=False)

### ID Staff Members

In [ ]:
df[df.name =='Staff']

### ID students accessing curriculumn pass the end date of program

In [ ]:
students= df[df.name !='Staff']

In [ ]:
peeking_after = students[students.end_date<students.index]
peeking_after

### Use Bollinger Band method to ID students with anomalous activity. (set k=2)

In [ ]:
def acquire_logs(user=env.user, password=env.password, host=env.host):
    '''
    This function queries the Codeup MySQL curriculum_logs database and returns a dataframe
    '''
    url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
    query = '''
    SELECT date,
           path ,
           user_id,
           cohort_id,
           ip as source_ip
    FROM logs;
    '''
    df = pd.read_sql(query, url)
    return df

In [ ]:
def one_user_df_prep(df, user):
    '''
    This function returns a dataframe consisting of data for only a single defined user
    '''
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages_one_user = df['path'].resample('d').count()
    return pages_one_user

In [ ]:
def compute_pct_b(pages_one_user, span, weight, user):
    '''
    This function adds the %b of a bollinger band range for the page views of a single user's log activity
    '''
    # Calculate upper and lower bollinger band
    midband = pages_one_user.ewm(span=span).mean()
    stdev = pages_one_user.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    
    # Add upper and lower band values to dataframe
    bb = pd.concat([ub, lb], axis=1)
    
    # Combine all data into a single dataframe
    my_df = pd.concat([pages_one_user, midband, bb], axis=1)
    my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
    
    # Calculate percent b and relevant user id to dataframe
    my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

In [ ]:
def plot_bands(my_df, user):
    '''
    This function plots the bolliger bands of the page views for a single user
    '''
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages_one_user, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

In [ ]:
def find_anomalies(df, user, span, weight, plot=False):
    '''
    This function returns the records where a user's daily activity exceeded the upper limit of a bollinger band range
    '''
    
    # Reduce dataframe to represent a single user
    pages_one_user = one_user_df_prep(df, user)
    
    # Add bollinger band data to dataframe
    my_df = compute_pct_b(pages_one_user, span, weight, user)
    
    # Plot data if requested (plot=True)
    if plot:
        plot_bands(my_df, user)
    
    # Return only records that sit outside of bollinger band upper limit
    return my_df[my_df.pct_b>1]

In [ ]:
span = 30
weight = 2

anomalies = pd.DataFrame()
for u in list(students.user_id.unique()):
    user_df = find_anomalies(students, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [ ]:
anomalies

In [ ]:
anomalies.user_id.value_counts()

### Plot Users

In [ ]:
user = 1
span = 30
weight = 2
user_df = find_anomalies(df, user, span, weight, plot=True)

anomalies = pd.DataFrame()
user_df = find_anomalies(df, user, span, weight)
anomalies = pd.concat([anomalies, user_df], axis=0)